In [ ]:
import numpy as np
import sklearn
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.random_projection import GaussianRandomProjection
from sklearn.base import ClusterMixin
from sklearn.manifold import TSNE

import torch
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from tqdm import tqdm
import time

import seaborn as sns

In [ ]:
# set random seed
seed = 11
np.random.seed(seed)
torch.manual_seed(seed)
device = 'cpu'

In [ ]:
datasets = ['bean', 'fruit']

# Load bean dataset
bean = pd.read_excel('.//datasets//bean//Dry_Bean_Dataset.xlsx')

X1 = bean.iloc[:, :-1]
y1 = bean.iloc[:, -1]

encoder = preprocessing.LabelEncoder().fit(y1)
y1 = encoder.transform(y1)

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=seed, stratify=y1)

scaler1 = preprocessing.StandardScaler().fit(X1_train)
X1_train = scaler1.transform(X1_train)
X1_test = scaler1.transform(X1_test)

X1_train = X1_train.astype(np.float32)
X1_test = X1_test.astype(np.float32)
y1_train = y1_train.astype(np.int64)
y1_test = y1_test.astype(np.int64)



# fruit dataset
fruit = pd.read_excel('.//datasets//fruit//fruit.xlsx')

X2 = fruit.iloc[:, :-1]
y2 = fruit.iloc[:, -1]

encoder = preprocessing.LabelEncoder()
encoder.fit(y2)
y2 = encoder.transform(y2)

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=seed, stratify=y2)

scaler2 = preprocessing.StandardScaler().fit(X2_train)
X2_train = scaler2.transform(X2_train)
X2_test = scaler2.transform(X2_test)

X2_train = X2_train.astype(np.float32)
X2_test = X2_test.astype(np.float32)
y2_train = y2_train.astype(np.int64)
y2_test = y2_test.astype(np.int64)


In [ ]:
# data correlation matrices
# the following code is adapted from https://seaborn.pydata.org/examples/many_pairwise_correlations.html

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
plt.rcParams.update({'font.size': 14})

d = pd.DataFrame(data=X1_train, columns=bean.columns[:-1])
corr1 = d.corr()
mask = np.triu(np.ones_like(corr1, dtype=bool))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr1, mask=mask, cmap=cmap, vmin=-1, vmax=1, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5}, ax=ax, xticklabels=True, yticklabels=True)
ax.tick_params(axis='both', which='major', labelsize=12)
# plt.savefig('data_correlations1.png', dpi=500, bbox_inches='tight')

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
d = pd.DataFrame(data=X2_train, columns=fruit.columns[:-1])
corr2 = d.corr()
mask = np.triu(np.ones_like(corr2, dtype=bool))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr2, mask=mask, cmap=cmap, vmin=-1, vmax=1, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5}, ax=ax, xticklabels=True, yticklabels=True)
ax.tick_params(labelsize=12)

plt.tight_layout()
# plt.savefig('data_correlations2.png', dpi=500, bbox_inches='tight')

In [ ]:
# analyze which features are redudant based on correlation matrix
# the following code is adapted from https://github.com/adityav95/variable_reduction_correlation/blob/master/variable_reduction_by_correlation.ipynb

corr_coeff = 0.8
corr_matrix = corr1.abs()
corr_mean = corr_matrix.mean()
features_drop_list = [] # This will contain the list of features to be dropped
features_index_drop_list = [] # This will contain the index of features to be dropped as per df_input
corr_matrix = abs(corr_matrix)

# Selecting features to be dropped (Using two for loops that runs on one triangle of the corr_matrix to avoid checking the correlation of a variable with itself)
for i in range(corr_matrix.shape[0]):
    for j in range(i+1,corr_matrix.shape[0]):

        # The following if statement checks if each correlation value is higher than threshold (or equal) and also ensures the two columns have NOT been dropped already.  
        if corr_matrix.iloc[i,j]>=corr_coeff and i not in features_index_drop_list and j not in features_index_drop_list:
        
            # The following if statement checks which of the 2 variables with high correlation has a lower correlation with target and then drops it. If equal we can drop any and it drops the first one (This is arbitrary)
            if corr_mean[corr_matrix.columns[i]] >= corr_mean[corr_matrix.columns[j]]:
                features_drop_list.append(corr_matrix.columns[i])	# Name of variable that needs to be dropped appended to list
                features_index_drop_list.append(i)	# Index of variable that needs to be dropped appended to list. This is used to not check for the same variables repeatedly
            else:
                features_drop_list.append(corr_matrix.columns[j])
                features_index_drop_list.append(j)
print('Bean dataset: redudant features are: {}'.format(features_drop_list))
print('Bean dataset: # of redudant features is: {}'.format(len(features_drop_list)))
print('Bean dataset: # of remaining features is: {}'.format(X1_train.shape[1] - len(features_drop_list)))


corr_matrix = corr2.abs()
corr_mean = corr_matrix.mean()
features_drop_list = [] # This will contain the list of features to be dropped
features_index_drop_list = [] # This will contain the index of features to be dropped as per df_input
corr_matrix = abs(corr_matrix)

# Selecting features to be dropped (Using two for loops that runs on one triangle of the corr_matrix to avoid checking the correlation of a variable with itself)
for i in range(corr_matrix.shape[0]):
    for j in range(i+1,corr_matrix.shape[0]):

        # The following if statement checks if each correlation value is higher than threshold (or equal) and also ensures the two columns have NOT been dropped already.  
        if corr_matrix.iloc[i,j]>=corr_coeff and i not in features_index_drop_list and j not in features_index_drop_list:
        
            # The following if statement checks which of the 2 variables with high correlation has a lower correlation with target and then drops it. If equal we can drop any and it drops the first one (This is arbitrary)
            if corr_mean[corr_matrix.columns[i]] >= corr_mean[corr_matrix.columns[j]]:
                features_drop_list.append(corr_matrix.columns[i])	# Name of variable that needs to be dropped appended to list
                features_index_drop_list.append(i)	# Index of variable that needs to be dropped appended to list. This is used to not check for the same variables repeatedly
            else:
                features_drop_list.append(corr_matrix.columns[j])
                features_index_drop_list.append(j)
print('Fruit dataset: redudant features are: {}'.format(features_drop_list))
print('Fruit dataset: # of redudant features is: {}'.format(len(features_drop_list)))
print('Fruit dataset: # of remaining features is: {}'.format(X2_train.shape[1] - len(features_drop_list)))

In [ ]:
# analyzing clusters

for i in range(2):
    cluster_results_ch = {}
    cluster_results_time = {}
    
    if i == 0:
        X_train = X1_train
        y_train = y1_train
        X_test = X1_test
        y_test = y1_test
        file = 'set1'
    else:
        X_train = X2_train
        y_train = y2_train
        X_test = X2_test
        y_test = y2_test
        file = 'set2'
    
    k_max = 10

    for cov in ['full', 'tied', 'diag', 'spherical']:
        silh_list_gmm_cov1 = []
        silh_list_gmm_cov2 = []
        silh_list_gmm_cov3 = []
        for k in tqdm(range(2, k_max+1), desc='Running GMM clusters on '+datasets[i]+' dataset'):
            gmm = GaussianMixture(n_components=k, random_state=seed, max_iter=500, n_init=100, covariance_type=cov, tol=0.0001)
            t1 = time.time()
            gmm.fit(X_train)
            t2 = time.time()
            y_train_pred = gmm.predict(X_train)
            silh_list_gmm_cov1.append(metrics.calinski_harabasz_score(X_train, y_train_pred))
            silh_list_gmm_cov3.append(t2-t1)
        cluster_results_ch['GMM_'+cov] = silh_list_gmm_cov1
        cluster_results_time['GMM_'+cov] = silh_list_gmm_cov3
            
    
    silh_list_kmean1 = []
    silh_list_kmean3 = []
    for k in tqdm(range(2, k_max+1), desc='Running k-means clusters on '+datasets[i]+' dataset'):
        kmean = KMeans(n_clusters=k, max_iter=500, random_state=seed, n_init=500)
        t1 = time.time()
        kmean.fit(X_train)
        t2 = time.time()
        y_train_pred = kmean.predict(X_train)
        silh_list_kmean1.append(metrics.calinski_harabasz_score(X_train, y_train_pred))
        silh_list_kmean3.append(t2-t1)
    cluster_results_ch['KM'] = silh_list_kmean1
    cluster_results_time['KM'] = silh_list_kmean3

    cluster_results_ch = pd.DataFrame(cluster_results_ch).to_excel('cluster_ch_'+file+'.xlsx')
    cluster_results_time = pd.DataFrame(cluster_results_time).to_excel('cluster_time_'+file+'.xlsx')

In [ ]:
# clustering results

k_max = 10
set1 = pd.read_excel('cluster_ch_set1.xlsx', index_col=0)
set2 = pd.read_excel('cluster_ch_set2.xlsx', index_col=0)

fig, ax = plt.subplots(1, 2, figsize=(12, 5.5))
ax = ax.flatten()
ks = list(range(2, k_max+1))

ax[0].plot(ks, set1['GMM_full'], label='GMM full')
ax[0].plot(ks, set1['KM'], label='K-Means')
ax[0].plot(ks, set1['GMM_tied'], '--', label='GMM tied', alpha=0.4)
ax[0].plot(ks, set1['GMM_diag'], '--', label='GMM diag', alpha=0.4)
ax[0].plot(ks, set1['GMM_spherical'], '--', label='GMM spherical', alpha=0.4)
ax[0].set_xlabel('Number of clusters')
ax[0].set_ylabel('Calinski-Harabasz score')
ax[0].legend()

ax[1].plot(ks, set2['GMM_full'], label='GMM full')
ax[1].plot(ks, set2['KM'], label='K-Means')
ax[1].plot(ks, set2['GMM_tied'], '--', label='GMM tied', alpha=0.4)
ax[1].plot(ks, set2['GMM_diag'], '--', label='GMM diag', alpha=0.4)
ax[1].plot(ks, set2['GMM_spherical'], '--', label='GMM spherical', alpha=0.4)
ax[1].set_xlabel('Number of clusters')
ax[1].set_ylabel('Calinski-Harabasz score')
ax[1].legend()

plt.rcParams.update({'font.size': 14})
plt.tight_layout()
# plt.savefig('Clustering_k.png', bbox_inches='tight', dpi=500)

In [ ]:
# bean dataset 2 vs 5 clusters
X_train = X1_train
fig, ax = plt.subplots(1, 3, figsize=(12, 5.5))
ax = ax.flatten()
k = [2, 5]

for i in range(len(k)):
    km = KMeans(n_clusters=k[i], max_iter=500, random_state=seed, n_init=500)
    km.fit(X_train)
    y_train_pred = km.predict(X_train)
    
    # following code adapted from https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html
    silhouette_avg = metrics.silhouette_score(X_train, y_train_pred)
    sample_silhouette_values = metrics.silhouette_samples(X_train, y_train_pred)
        
    y_lower = 10
    for idx in range(k[i]):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[y_train_pred == idx]
    
        ith_cluster_silhouette_values.sort()
    
        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
    
        color = cm.nipy_spectral(float(idx) / k[i])
        ax[i].fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )
    
        # Label the silhouette plots with their cluster numbers at the middle
        ax[i].text(-0.05, y_lower + 0.5 * size_cluster_i, str(idx))
    
        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples
    
    ax[i].set_xlabel("Silhouette coefficient")
    ax[i].set_ylabel("Cluster label")
    
    # The vertical line for average silhouette score of all the values
    ax[i].axvline(x=silhouette_avg, color="red", linestyle="--")
    
    ax[i].set_yticks([])  # Clear the yaxis labels / ticks
    ax[i].set_xticks([-0.2, 0, 0.2, 0.4, 0.6, 0.8])




X_train = X2_train

km = KMeans(n_clusters=3, max_iter=500, random_state=seed, n_init=500)
km.fit(X_train)
y_train_pred = km.predict(X_train)

# following code adapted from https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html
silhouette_avg = metrics.silhouette_score(X_train, y_train_pred)
sample_silhouette_values = metrics.silhouette_samples(X_train, y_train_pred)
    
y_lower = 10
for idx in range(3):
    # Aggregate the silhouette scores for samples belonging to
    # cluster i, and sort them
    ith_cluster_silhouette_values = sample_silhouette_values[y_train_pred == idx]

    ith_cluster_silhouette_values.sort()

    size_cluster_i = ith_cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster_i

    color = cm.nipy_spectral(float(idx) / 3)
    ax[2].fill_betweenx(
        np.arange(y_lower, y_upper),
        0,
        ith_cluster_silhouette_values,
        facecolor=color,
        edgecolor=color,
        alpha=0.7,
    )

    # Label the silhouette plots with their cluster numbers at the middle
    ax[2].text(-0.05, y_lower + 0.5 * size_cluster_i, str(idx))

    # Compute the new y_lower for next plot
    y_lower = y_upper + 10  # 10 for the 0 samples

ax[2].set_xlabel("Silhouette coefficient")
ax[2].set_ylabel("Cluster label")

# The vertical line for average silhouette score of all the values
ax[2].axvline(x=silhouette_avg, color="red", linestyle="--")

ax[2].set_yticks([])  # Clear the yaxis labels / ticks
ax[2].set_xticks([-0.2, 0, 0.2, 0.4, 0.6, 0.8])


plt.rcParams.update({'font.size': 14})
plt.tight_layout()
# plt.savefig('Clustering_bean_k.png', bbox_inches='tight', dpi=500)

In [ ]:
# clustering visualization with tsne

fig, ax = plt.subplots(2, 2, figsize=(12, 10.5))
ax = ax.flatten()
k = {'bean_gmm': 5, 'bean_km': 5, 'fruit_gmm': 3, 'fruit_km': 3}
perplexities = [50, 50]

cluster_labels = []
for i in range(2):
    if i == 0:
        X_train = X1_train
        y_train = y1_train
        cov = 'full'
    else:
        X_train = X2_train
        y_train = y2_train
        cov = 'tied'
    
    gmm = GaussianMixture(n_components=k[datasets[i]+'_gmm'], random_state=seed, max_iter=500, n_init=25, covariance_type='full', tol=0.0001)
    gmm.fit(X_train)
    y_train_pred = gmm.predict(X_train)
    vms = metrics.homogeneity_completeness_v_measure(y_train, y_train_pred)
    print('{} dataset with GMM clustering, the homogeneity is {}'.format(datasets[i], vms[0]))
    print('{} dataset with GMM clustering, the completeness is {}'.format(datasets[i], vms[1]))
    print('{} dataset with GMM clustering, the V measure is {}'.format(datasets[i], vms[2]))

    ordering = np.argsort(np.unique(y_train_pred, return_counts=True)[1])

    tsne = TSNE(n_components=2, perplexity=perplexities[i])
    proj = tsne.fit_transform(X_train)

    for j in ordering:
        ax[2*i].plot(proj[y_train_pred==j, 0], proj[y_train_pred==j, 1], '.', label='cluster '+str(j))
    ax[2*i].set_xlabel('TC1')
    ax[2*i].set_ylabel('TC2')

    cluster_label = np.zeros((k[datasets[i]+'_gmm'], len(np.unique(y_train))))
    for cl in range(cluster_label.shape[0]):
        for lb in range(cluster_label.shape[1]):
            cluster_label[cl, lb] = np.count_nonzero((y_train_pred==ordering[cl]) & (y_train==lb))
    cluster_label = pd.DataFrame(cluster_label, index=np.arange(k[datasets[i]+'_gmm']), columns=np.unique(y_train))
    cluster_labels.append(cluster_label)

    km = KMeans(n_clusters=k[datasets[i]+'_km'], max_iter=500, random_state=seed, n_init=500)
    km.fit(X_train)
    y_train_pred = km.predict(X_train)
    vms = metrics.homogeneity_completeness_v_measure(y_train, y_train_pred)
    print('{} dataset with KM clustering, the homogeneity is {}'.format(datasets[i], vms[0]))
    print('{} dataset with KM clustering, the completeness is {}'.format(datasets[i], vms[1]))
    print('{} dataset with KM clustering, the V measure is {}'.format(datasets[i], vms[2]))
    
    ordering = np.argsort(np.unique(y_train_pred, return_counts=True)[1])
    
    for j in ordering:
        ax[2*i+1].plot(proj[y_train_pred==j, 0], proj[y_train_pred==j, 1], '.', label='cluster '+str(j))
    ax[2*i+1].set_xlabel('TC1')
    ax[2*i+1].set_ylabel('TC2')

    cluster_label = np.zeros((k[datasets[i]+'_km'], len(np.unique(y_train))))
    for cl in range(cluster_label.shape[0]):
        for lb in range(cluster_label.shape[1]):
            cluster_label[cl, lb] = np.count_nonzero((y_train_pred==ordering[cl]) & (y_train==lb))
    cluster_label = pd.DataFrame(cluster_label, index=np.arange(k[datasets[i]+'_km']), columns=np.unique(y_train))
    cluster_labels.append(cluster_label)
    
plt.rcParams.update({'font.size': 14})
plt.tight_layout()
# plt.savefig('Clustering_vis.png', bbox_inches='tight', dpi=500)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 10.5))
ax = ax.flatten()

ax[0] = sns.heatmap(cluster_labels[0], cmap=cmap, center=0, square=True, linewidths=.5, vmin=0, vmax=2500, cbar_kws={"shrink": .5}, ax=ax[0])
ax[0].set_xlabel('Labels')
ax[0].set_ylabel('Clusters')

ax[1] = sns.heatmap(cluster_labels[1], cmap=cmap, center=0, square=True, linewidths=.5, vmin=0, vmax=2500, cbar_kws={"shrink": .5}, ax=ax[1])
ax[1].set_xlabel('Labels')
ax[1].set_ylabel('Clusters')

ax[2] = sns.heatmap(cluster_labels[2], cmap=cmap, center=0, square=True, linewidths=.5, vmin=0, vmax=200, cbar_kws={"shrink": .5}, ax=ax[2])
ax[2].set_xlabel('Labels')
ax[2].set_ylabel('Clusters')

ax[3] = sns.heatmap(cluster_labels[3], cmap=cmap, center=0, square=True, linewidths=.5, vmin=0, vmax=200, cbar_kws={"shrink": .5}, ax=ax[3])
ax[3].set_xlabel('Labels')
ax[3].set_ylabel('Clusters')

plt.rcParams.update({'font.size': 14})
plt.tight_layout()
# plt.savefig('Clustering_results.png', bbox_inches='tight', dpi=500)

In [ ]:
# PCA analysis

fig, ax = plt.subplots(1, 2, figsize=(12, 5.5))
ax = ax.flatten()

for i in range(2):
    if i == 0:
        X_train = X1_train
    else:
        X_train = X2_train

    pca = PCA(n_components=None, random_state=seed)
    pca.fit(X_train)
    cumvar = np.cumsum(pca.explained_variance_ratio_)
    ln1 = ax[i].plot(list(range(1, X_train.shape[1] + 1)), cumvar, label='Cumulative variance')

    thresh = 0.95
    comp_best = np.argmax(cumvar > thresh)
    
    ax[i].axhline(y=cumvar[comp_best], color='grey', linestyle='--')
    ax[i].axvline(x=comp_best+1, color='grey', linestyle='--')
    ax[i].set_xlabel('Number of PCs')
    ax[i].set_ylabel('Cumulative variance (%)')
    
    if i == 0:
        ax[i].set_xticks([0, 2, 4, 6, 8, 10, 12, 14, 16])
        ax[i].set_xlim([-0.5, 16.5])
        text_x = comp_best + 1.5
        text_y = thresh - 0.01
    else:
        ax[i].set_xticks([0, 5, 10, 15, 20, 25, 30, 35])
        ax[i].set_xlim([-0.5, 36])
        text_x = comp_best + 2
        text_y = thresh - 0.04
    ax[i].text(text_x, text_y, 'Components = '+str(comp_best+1), color = 'black')
    
    ax2 = ax[i].twinx()
    
    error = []
    for k in range(1, X_train.shape[1]+1):
        pca = PCA(n_components=k, random_state=seed)
        pca.fit(X_train)
        X_train_trans = pca.transform(X_train)
        X_train_recons = X_train_trans @ pca.components_
        error.append(metrics.mean_squared_error(X_train, X_train_recons))
    ln2 = ax2.plot(list(range(1, X_train.shape[1] + 1)), error, color='orange', label='Reconstruction RMSE')
    ax2.set_ylabel('Reconstruction RMSE')

lns = ln1 + ln2
labs = [l.get_label() for l in lns]
fig.legend(lns, labs, bbox_to_anchor =(0.5, -0.07), loc='lower center', ncol=2)
plt.rcParams.update({'font.size': 14})
plt.tight_layout()
# plt.savefig('PCA_k.png', bbox_inches='tight', dpi=500)

In [ ]:
# PCA analysis 2

fig, ax = plt.subplots(1, 2, figsize=(12, 7))
ax = ax.flatten()

X_train = X1_train
pca = PCA(n_components=5, random_state=seed)
pca.fit(X_train)
var = pca.explained_variance_ratio_
cols = ['PC{}'.format(i) for i in range(1, 6)]
pca_matrix = pd.DataFrame(pca.components_.T * var, index=X1.columns, columns=cols)
ax[0] = sns.heatmap(pca_matrix, cmap=cmap, center=0, square=True, linewidths=.5, vmin=-0.2, vmax=0.2, cbar_kws={"shrink": .5}, ax=ax[0],  xticklabels=True, yticklabels=True)

X_train = X2_train
pca = PCA(n_components=8, random_state=seed)
pca.fit(X_train)
var = pca.explained_variance_ratio_
cols = ['PC{}'.format(i) for i in range(1, 9)]
pca_matrix = pd.DataFrame(pca.components_.T * var, index=X2.columns, columns=cols)
ax[1] = sns.heatmap(pca_matrix, cmap=cmap, center=0, square=True, linewidths=.5, vmin=-0.11, vmax=0.11, cbar_kws={"shrink": .5}, ax=ax[1],  xticklabels=True, yticklabels=True)
ax[1].tick_params(labelsize=10)

plt.tight_layout()
# plt.savefig('PCA_analysis.png', dpi=500, bbox_inches='tight')

In [ ]:
# ICA analysis
seed = 7
fig, ax = plt.subplots(1, 2, figsize=(12, 5.5))
ax = ax.flatten()
comp_best = [3, 4]

for i in range(2):
    if i == 0:
        X_train = X1_train
    else:
        X_train = X2_train
    max_iter = 10000
    ica = FastICA(n_components=X_train.shape[1], random_state=seed, max_iter=max_iter)
    ica.fit(X_train)
    X_train_trans = ica.transform(X_train)
    kurtosis = pd.DataFrame(X_train_trans).kurt(axis=0).abs()
    kurtosis = pd.concat([pd.DataFrame(kurtosis, columns=['kurt']), pd.DataFrame(ica.components_)], axis=1)
    kurtosis = kurtosis.sort_values(by='kurt', ascending=False)

    cumkurt = np.cumsum(kurtosis['kurt'].to_numpy())
    cumkurt = cumkurt / np.sum(kurtosis['kurt'])

    thresh = 0.95
    comp_best = np.argmax(cumkurt > thresh)

    error = []
    for k in range(1, X_train.shape[1]+1):
        best_components = kurtosis.iloc[:k, 1:]
        best_idx = best_components.index
        X_train_trans_sel = X_train_trans.copy()
        to_remove = np.delete(np.arange(X_train.shape[1]), best_idx)
        X_train_trans_sel[:, to_remove] = 0
        X_train_recons = ica.inverse_transform(X_train_trans_sel)
        error.append(metrics.mean_squared_error(X_train, X_train_recons))

    if i == 0:
        ax[i].set_xticks([0, 2, 4, 6, 8, 10, 12, 14, 16])
        ax[i].set_xlim([-0.5, 16.5])
        text_x = comp_best + 1.5
        text_y = cumkurt[comp_best] - 0.06  
    else:
        ax[i].set_xticks([0, 5, 10, 15, 20, 25, 30, 35])
        ax[i].set_xlim([-0.5, 36])
        text_x = comp_best + 2.5
        text_y = cumkurt[comp_best] - 0.06    

    ax[i].axvline(x=comp_best+1, color='grey', linestyle='--')
    
    ln1 = ax[i].bar(list(range(1, X_train.shape[1]+1)), kurtosis['kurt'].tolist(), label='Kurtosis')
    ax[i].set_xlabel('IC or Number of ICs')
    ax[i].set_ylabel('Component kurtosis')
    ax2 = ax[i].twinx()
    ax2.axhline(y=cumkurt[comp_best], color='grey', linestyle='--')
    ax2.text(text_x, text_y, 'Components = '+str(comp_best), color = 'black')
    ln2 = ax2.plot(list(range(1, X_train.shape[1]+1)), error, color='orange', label='Reconstruction RMSE', linewidth=2)
    ln3 = ax2.plot(list(range(1, X_train.shape[1]+1)), cumkurt, color='#d62728', label='Cumulative kurtosis', linewidth=2)
    ax2.set_ylabel('Reconstruction RMSE or Cumulative kurtosis')

lns = [ln1] + ln2 + ln3
labs = [l.get_label() for l in lns]
fig.legend(lns, labs, bbox_to_anchor =(0.5, -0.07), loc='lower center', ncol=3)

plt.rcParams.update({'font.size': 14})
plt.tight_layout()
# plt.savefig('ICA_k.png', bbox_inches='tight', dpi=500)

In [ ]:
# ICA analysis 2

fig, ax = plt.subplots(1, 2, figsize=(12, 7))
ax = ax.flatten()

X_train = X1_train
ica = FastICA(n_components=None, random_state=seed)
ica.fit(X_train)
X_train_trans = ica.transform(X_train)
kurtosis = pd.DataFrame(X_train_trans).kurt(axis=0).abs()
kurtosis = pd.concat([pd.DataFrame(kurtosis, columns=['kurt']), pd.DataFrame(ica.components_)], axis=1)
kurtosis = kurtosis.sort_values(by='kurt', ascending=False)
best_components = kurtosis.iloc[:8, 1:].to_numpy()
best_components = (best_components.T / np.linalg.norm(best_components, axis=1)).T
best_kurtosis = kurtosis.iloc[:8, 0].to_numpy() / np.sum(kurtosis['kurt'])

cols = ['IC{}'.format(i) for i in range(1, 9)]
ica_matrix = best_components.T * best_kurtosis
ica_matrix = pd.DataFrame(ica_matrix, index=X1.columns, columns=cols)
ax[0] = sns.heatmap(ica_matrix, cmap=cmap, center=0, square=True, linewidths=.5, vmin=-0.22, vmax=0.22, cbar_kws={"shrink": .5}, ax=ax[0],  xticklabels=True, yticklabels=True)
ax[0].tick_params('x', labelrotation=90)

X_train = X2_train
ica = FastICA(n_components=None, random_state=seed)
ica.fit(X_train)
X_train_trans = ica.transform(X_train)
kurtosis = pd.DataFrame(X_train_trans).kurt(axis=0).abs()
kurtosis = pd.concat([pd.DataFrame(kurtosis, columns=['kurt']), pd.DataFrame(ica.components_)], axis=1)
kurtosis = kurtosis.sort_values(by='kurt', ascending=False)
best_components = kurtosis.iloc[:17, 1:].to_numpy()
best_components = (best_components.T / np.linalg.norm(best_components, axis=1)).T
best_kurtosis = kurtosis.iloc[:17, 0].to_numpy() / np.sum(kurtosis['kurt'])

cols = ['IC{}'.format(i) for i in range(1, 18)]
ica_matrix = best_components.T * best_kurtosis
ica_matrix = pd.DataFrame(ica_matrix, index=X2.columns, columns=cols)
ax[1] = sns.heatmap(ica_matrix, cmap=cmap, center=0, square=True, linewidths=.5, vmin=-0.15, vmax=0.15, cbar_kws={"shrink": .5}, ax=ax[1],  xticklabels=True, yticklabels=True)
ax[1].tick_params(labelsize=10)

plt.tight_layout()
# plt.savefig('ICA_analysis.png', dpi=500, bbox_inches='tight')

In [ ]:
seeds = [0, 7, 42, 69, 518]
comps = [6, 14]
fig, ax = plt.subplots(1, 2, figsize=(12, 5.5))
ax = ax.flatten()

for i in range(2):
    if i == 0:
        X_train = X1_train
    else:
        X_train = X2_train

    error_avg = []
    error_std = []
    for k in range(1, X_train.shape[1]+1):
        error = []
        for seed in seeds:
            # rp = SparseRandomProjection(n_components=k, random_state=seed, compute_inverse_components=True)
            rp = GaussianRandomProjection(n_components=k, random_state=seed, compute_inverse_components=True)
            rp.fit(X_train)
            X_train_trans = rp.transform(X_train)
            X_train_recons = X_train_trans @ rp.inverse_components_.T
            error.append(metrics.mean_squared_error(X_train, X_train_recons))
        error_avg.append(np.mean(error))
        error_std.append(np.std(error))  

    if i == 0:
        ax[i].set_xticks([0, 2, 4, 6, 8, 10, 12, 14, 16])
        ax[i].set_xlim([-0.5, 16.5])
        text_x = comps[i] + 0.5
    else:
        text_x = comps[i] + 1
    text_y = 0.8
    
    ax[i].plot(list(range(1, X_train.shape[1]+1)), error_avg)
    ax[i].fill_between(list(range(1, X_train.shape[1]+1)), np.array(error_avg) - np.array(error_std), np.array(error_avg) + np.array(error_std), alpha=0.1, color='darkorchid')
    ax[i].set_xlabel('Number of RP components')
    ax[i].set_ylabel('Reconstruction RMSE')
    # ax[i].axvline(x=comps[i], color='grey', linestyle='--')
    # ax[i].text(text_x, text_y, 'Components = '+str(comps[i]), color = 'black')
    # ax[i].axhline(y=error_avg[comps[i]-1], color='grey', linestyle='--')

plt.rcParams.update({'font.size': 14})
plt.tight_layout()
# plt.savefig('RP_k.png', bbox_inches='tight', dpi=500)

In [ ]:
cPCA = [5, 8]
cICA = [6, 10]
cRP = [6, 14]
seed = 8
for DR in ['PCA', 'ICA', 'RP']:
    for i in range(2):
        cluster_results_ch = {}
        cluster_results_time = {}
        
        if i == 0:
            X_train = X1_train
            file = 'set1'
        else:
            X_train = X2_train
            file = 'set2'

        if DR == 'PCA':
            DR_algo = PCA(n_components=cPCA[i], random_state=seed)
            DR_algo.fit(X_train)
            X_train = DR_algo.transform(X_train)
        elif DR == 'ICA':
            DR_algo = FastICA(n_components=cICA[i], random_state=seed, max_iter=2000)
            DR_algo.fit(X_train)
            
            X_train = DR_algo.transform(X_train)
        elif DR == 'RP':
            DR_algo = GaussianRandomProjection(n_components=cRP[i], random_state=seed)
            DR_algo.fit(X_train)
            X_train = DR_algo.transform(X_train)

        k_max = 10
        
        for cov in ['full', 'tied']:
            silh_list_gmm_cov1 = []
            silh_list_gmm_cov3 = []
            for k in tqdm(range(2, k_max+1), desc='Running GMM clusters on '+datasets[i]+' dataset with '+DR+' transformation'):
                gmm = GaussianMixture(n_components=k, random_state=seed, max_iter=500, n_init=100, covariance_type=cov, tol=0.0001)
                t1 = time.time()
                gmm.fit(X_train)
                t2 = time.time()
                y_train_pred = gmm.predict(X_train)
                silh_list_gmm_cov1.append(metrics.calinski_harabasz_score(X_train, y_train_pred))
                silh_list_gmm_cov3.append(t2-t1)
            cluster_results_ch['GMM_'+cov] = silh_list_gmm_cov1
            cluster_results_time['GMM_'+cov] = silh_list_gmm_cov3
                
        
        silh_list_kmean1 = []
        silh_list_kmean3 = []
        for k in tqdm(range(2, k_max+1), desc='Running k-means clusters on '+datasets[i]+' dataset with '+DR+' transformation'):
            kmean = KMeans(n_clusters=k, max_iter=500, random_state=seed, n_init=500)
            t1 = time.time()
            kmean.fit(X_train)
            t2 = time.time()
            y_train_pred = kmean.predict(X_train)
            silh_list_kmean1.append(metrics.calinski_harabasz_score(X_train, y_train_pred))
            silh_list_kmean3.append(t2-t1)
        cluster_results_ch['KM'] = silh_list_kmean1
        cluster_results_time['KM'] = silh_list_kmean3
    
        cluster_results_ch = pd.DataFrame(cluster_results_ch).to_excel('cluster_ch_'+DR+file+'.xlsx')
        cluster_results_time = pd.DataFrame(cluster_results_time).to_excel('cluster_time_'+DR+file+'.xlsx')

In [ ]:
# clustering results

k_max = 10
PCAset1 = pd.read_excel('cluster_ch_PCAset1.xlsx', index_col=0)
PCAset2 = pd.read_excel('cluster_ch_PCAset2.xlsx', index_col=0)
ICAset1 = pd.read_excel('cluster_ch_ICAset1.xlsx', index_col=0)
ICAset2 = pd.read_excel('cluster_ch_ICAset2.xlsx', index_col=0)
RPset1 = pd.read_excel('cluster_ch_RPset1.xlsx', index_col=0)
RPset2 = pd.read_excel('cluster_ch_RPset2.xlsx', index_col=0)

fig, ax = plt.subplots(3, 2, figsize=(12, 16))
ax = ax.flatten()
ks = list(range(2, k_max+1))

ax[0].plot(ks, PCAset1['GMM_full'], label='GMM')
ax[0].plot(ks, PCAset1['KM'], label='K-Means')
ax[0].set_xlabel('Number of clusters')
ax[0].set_ylabel('Calinski-Harabasz score')
ax[0].legend()

ax[1].plot(ks, PCAset2['GMM_full'], label='GMM')
ax[1].plot(ks, PCAset2['KM'], label='K-Means')
ax[1].set_xlabel('Number of clusters')
ax[1].set_ylabel('Calinski-Harabasz score')
ax[1].legend()

ax[2].plot(ks, ICAset1['GMM_full'], label='GMM')
ax[2].plot(ks, ICAset1['KM'], label='K-Means')
ax[2].set_xlabel('Number of clusters')
ax[2].set_ylabel('Calinski-Harabasz score')
ax[2].legend()

ax[3].plot(ks, ICAset2['GMM_full'], label='GMM')
ax[3].plot(ks, ICAset2['KM'], label='K-Means')
ax[3].set_xlabel('Number of clusters')
ax[3].set_ylabel('Calinski-Harabasz score')
ax[3].legend()

ax[4].plot(ks, RPset1['GMM_full'], label='GMM')
ax[4].plot(ks, RPset1['KM'], label='K-Means')
ax[4].set_xlabel('Number of clusters')
ax[4].set_ylabel('Calinski-Harabasz score')
ax[4].legend()

ax[5].plot(ks, RPset2['GMM_full'], label='GMM')
ax[5].plot(ks, RPset2['KM'], label='K-Means')
ax[5].set_xlabel('Number of clusters')
ax[5].set_ylabel('Calinski-Harabasz score')
ax[5].legend()

plt.rcParams.update({'font.size': 14})
plt.tight_layout()
# plt.savefig('Clustering_afterDR_k.png', bbox_inches='tight', dpi=500)

In [ ]:
fig, ax = plt.subplots(6, 2, figsize=(12, 31.5))
ax = ax.flatten()
k_PCA_GMM = [5, 4]
k_ICA_GMM = [6, 4]
k_PCA_KM = [5, 3]
k_ICA_KM = [7, 5]
k_RP_GMM = [5, 3]
k_RP_KM = [5, 3]
perplexities = [70, 70]
cPCA = [5, 8]
cICA = [6, 10]
cRP = [6, 14]

m = 0
cluster_labels = []
for i in range(2):
    for DR in ['PCA', 'ICA', 'RP']:
        if i == 0:
            X_train = X1_train
            y_train = y1_train
            if DR == 'PCA':
                cov = 'full'
            elif DR == 'ICA':
                cov = 'tied'
        else:
            X_train = X2_train
            y_train = y2_train
            if DR == 'PCA':
                cov = 'full'
            elif DR == 'ICA':
                cov = 'tied'

        if DR == 'PCA':
            DR_algo = PCA(n_components=cPCA[i], random_state=seed)
            DR_algo.fit(X_train)
            X_train = DR_algo.transform(X_train)
            k_GMM = k_PCA_GMM
            k_KM = k_PCA_KM
        elif DR == 'ICA':
            DR_algo = FastICA(n_components=cICA[i], random_state=seed, max_iter=2000)
            DR_algo.fit(X_train)
            X_train = DR_algo.transform(X_train)
            
            k_GMM = k_ICA_GMM
            k_KM = k_ICA_KM

        elif DR == 'RP':
            DR_algo = GaussianRandomProjection(n_components=cRP[i], random_state=seed)
            DR_algo.fit(X_train)
            X_train = DR_algo.transform(X_train)
            k_GMM = k_RP_GMM
            k_KM = k_RP_KM
        
        gmm = GaussianMixture(n_components=k_GMM[i], random_state=seed, max_iter=500, n_init=25, covariance_type=cov, tol=0.0001)
        gmm.fit(X_train)
        y_train_pred = gmm.predict(X_train)

        vms = metrics.homogeneity_completeness_v_measure(y_train, y_train_pred)
        print('{} dataset after {} with GMM clustering, the homogeneity is {}'.format(datasets[i], DR, vms[0]))
        print('{} dataset after {} with GMM clustering, the completeness is {}'.format(datasets[i], DR, vms[1]))
        print('{} dataset after {} with GMM clustering, the V measure is {}'.format(datasets[i], DR, vms[2]))
    
        ordering = np.argsort(np.unique(y_train_pred, return_counts=True)[1])

        cluster_label = np.zeros((k_GMM[i], len(np.unique(y_train))))
        for cl in range(cluster_label.shape[0]):
            for lb in range(cluster_label.shape[1]):
                cluster_label[cl, lb] = np.count_nonzero((y_train_pred==ordering[cl]) & (y_train==lb))
        cluster_label = pd.DataFrame(cluster_label, index=np.arange(k_GMM[i]), columns=np.unique(y_train))
        cluster_labels.append(cluster_label)

        tsne = TSNE(n_components=2, perplexity=perplexities[i])
        proj = tsne.fit_transform(X_train)
        
        for j in ordering:
            ax[m].plot(proj[y_train_pred==j, 0], proj[y_train_pred==j, 1], '.', label='cluster '+str(j))
        ax[m].set_xlabel('TC1')
        ax[m].set_ylabel('TC2')
    
        km = KMeans(n_clusters=k_KM[i], max_iter=500, random_state=seed, n_init=500)
        km.fit(X_train)
        y_train_pred = km.predict(X_train)

        vms = metrics.homogeneity_completeness_v_measure(y_train, y_train_pred)
        print('{} dataset after {} with KM clustering, the homogeneity is {}'.format(datasets[i], DR, vms[0]))
        print('{} dataset after {} with KM clustering, the completeness is {}'.format(datasets[i], DR, vms[1]))
        print('{} dataset after {} with KM clustering, the V measure is {}'.format(datasets[i], DR, vms[2]))
        
        ordering = np.argsort(np.unique(y_train_pred, return_counts=True)[1])

        cluster_label = np.zeros((k_KM[i], len(np.unique(y_train))))
        for cl in range(cluster_label.shape[0]):
            for lb in range(cluster_label.shape[1]):
                cluster_label[cl, lb] = np.count_nonzero((y_train_pred==ordering[cl]) & (y_train==lb))
        cluster_label = pd.DataFrame(cluster_label, index=np.arange(k_KM[i]), columns=np.unique(y_train))
        cluster_labels.append(cluster_label)
        
        for j in ordering:
            ax[m+1].plot(proj[y_train_pred==j, 0], proj[y_train_pred==j, 1], '.', label='cluster '+str(j))
        ax[m+1].set_xlabel('TC1')
        ax[m+1].set_ylabel('TC2')

        m += 2

plt.rcParams.update({'font.size': 14})
plt.tight_layout()
# plt.savefig('Clustering_afterDR_vis.png', bbox_inches='tight', dpi=500)

In [ ]:
fig, ax = plt.subplots(6, 2, figsize=(12, 25))
ax = ax.flatten()

ax[0] = sns.heatmap(cluster_labels[0], cmap=cmap, center=0, square=True, linewidths=.5, vmin=0, vmax=2200, cbar_kws={"shrink": .5}, ax=ax[0])
ax[0].set_xlabel('Labels')
ax[0].set_ylabel('Clusters')

ax[1] = sns.heatmap(cluster_labels[1], cmap=cmap, center=0, square=True, linewidths=.5, vmin=0, vmax=2200, cbar_kws={"shrink": .5}, ax=ax[1])
ax[1].set_xlabel('Labels')
ax[1].set_ylabel('Clusters')

ax[2] = sns.heatmap(cluster_labels[2], cmap=cmap, center=0, square=True, linewidths=.5, vmin=0, vmax=2200, cbar_kws={"shrink": .5}, ax=ax[2])
ax[2].set_xlabel('Labels')
ax[2].set_ylabel('Clusters')

ax[3] = sns.heatmap(cluster_labels[3], cmap=cmap, center=0, square=True, linewidths=.5, vmin=0, vmax=2200, cbar_kws={"shrink": .5}, ax=ax[3])
ax[3].set_xlabel('Labels')
ax[3].set_ylabel('Clusters')

ax[4] = sns.heatmap(cluster_labels[4], cmap=cmap, center=0, square=True, linewidths=.5, vmin=0, vmax=2200, cbar_kws={"shrink": .5}, ax=ax[4])
ax[4].set_xlabel('Labels')
ax[4].set_ylabel('Clusters')

ax[5] = sns.heatmap(cluster_labels[5], cmap=cmap, center=0, square=True, linewidths=.5, vmin=0, vmax=2200, cbar_kws={"shrink": .5}, ax=ax[5])
ax[5].set_xlabel('Labels')
ax[5].set_ylabel('Clusters')

ax[6] = sns.heatmap(cluster_labels[6], cmap=cmap, center=0, square=True, linewidths=.5, vmin=0, vmax=160, cbar_kws={"shrink": .5}, ax=ax[6])
ax[6].set_xlabel('Labels')
ax[6].set_ylabel('Clusters')

ax[7] = sns.heatmap(cluster_labels[7], cmap=cmap, center=0, square=True, linewidths=.5, vmin=0, vmax=160, cbar_kws={"shrink": .5}, ax=ax[7])
ax[7].set_xlabel('Labels')
ax[7].set_ylabel('Clusters')

ax[8] = sns.heatmap(cluster_labels[8], cmap=cmap, center=0, square=True, linewidths=.5, vmin=0, vmax=160, cbar_kws={"shrink": .5}, ax=ax[8])
ax[8].set_xlabel('Labels')
ax[8].set_ylabel('Clusters')

ax[9] = sns.heatmap(cluster_labels[9], cmap=cmap, center=0, square=True, linewidths=.5, vmin=0, vmax=160, cbar_kws={"shrink": .5}, ax=ax[9])
ax[9].set_xlabel('Labels')
ax[9].set_ylabel('Clusters')

ax[10] = sns.heatmap(cluster_labels[10], cmap=cmap, center=0, square=True, linewidths=.5, vmin=0, vmax=160, cbar_kws={"shrink": .5}, ax=ax[10])
ax[10].set_xlabel('Labels')
ax[10].set_ylabel('Clusters')

ax[11] = sns.heatmap(cluster_labels[11], cmap=cmap, center=0, square=True, linewidths=.5, vmin=0, vmax=160, cbar_kws={"shrink": .5}, ax=ax[11])
ax[11].set_xlabel('Labels')
ax[11].set_ylabel('Clusters')

plt.rcParams.update({'font.size': 14})
plt.tight_layout()
plt.savefig('Clustering_afterDR_results.png', bbox_inches='tight', dpi=500)